In [1]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, classification_report

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'omar-vargas-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'telecom_churn.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
churn_data = pd.read_csv(file_content_stream)
churn_data.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [3]:
## Defining input and target variables
X = churn_data[['AccountWeeks', 'ContractRenewal', 'CustServCalls', 'DayMins', 'MonthlyCharge']]
Y = churn_data['Churn']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [4]:
## Running smote
X_smote, Y_smote = SMOTE().fit_resample(X_train, Y_train)

Random Forest

In [5]:
## Building the model
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_smote, Y_smote)

## Predicting on the test
RF_pred = RF_md.predict_proba(X_test)[:, 1]

## ROC
fpr, tpr, threshold = roc_curve(Y_test, RF_pred)

## Finding the optimal cutoff
RF_cutoff = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Cutoff': threshold})

## Computing the distance to perfect model
RF_cutoff['Distance'] = np.sqrt(RF_cutoff['FPR']**2 + (1 - RF_cutoff['TPR'])**2)

## Sorting
RF_cutoff = RF_cutoff.sort_values(by = 'Distance').reset_index(drop = True)

## Changing likelihoods to labels
RF_pred_label = np.where(RF_pred < RF_cutoff['Cutoff'][0], 0, 1)

## Classification report
print(classification_report(Y_test, RF_pred_label))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       570
           1       0.54      0.89      0.67        97

    accuracy                           0.87       667
   macro avg       0.76      0.88      0.80       667
weighted avg       0.91      0.87      0.89       667



AdaBoost

In [6]:
## Building the model
Ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_smote, Y_smote)

## Predicting on the test
Ada_pred = Ada_md.predict_proba(X_test)[:, 1]

## ROC
fpr, tpr, threshold = roc_curve(Y_test, Ada_pred)

## Finding the optimal cutoff
Ada_cutoff = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Cutoff': threshold})

## Computing the distance to perfect model
Ada_cutoff['Distance'] = np.sqrt(Ada_cutoff['FPR']**2 + (1 - Ada_cutoff['TPR']**2))

## Sorting
Ada_cutoff = Ada_cutoff.sort_values(by = 'Distance').reset_index(drop = True)

## Changing likelihoods to labels
Ada_pred_label = np.where(Ada_pred < Ada_cutoff['Cutoff'][0], 0, 1)

## Classification report
print(classification_report(Y_test, Ada_pred_label))

              precision    recall  f1-score   support

           0       0.97      0.90      0.94       570
           1       0.59      0.86      0.70        97

    accuracy                           0.89       667
   macro avg       0.78      0.88      0.82       667
weighted avg       0.92      0.89      0.90       667



In [ ]:
## Based on my results, I would use AdaBoost to predict customer churn.